# Discrimination of biological sex from urinary metabolic profiles using Partial Least Squares – Discriminant Analysis

In this notebook we will use a PLS-DA classifier model to discriminate urine biofluid metabolic profiles based on the participant's sex. 

The notebook is divided in the following steps:

1) Model fitting basics: Fit a PLS-DA model to predict biological sex from the metabolic profile data, using different types of scaling.
2) Model cross-validation and parameter selection: Describe model cross-validation, robust parameter selection with double-cross validation and performance assessment for a PLS-DA model.

3) Model interpretation: Inspect the variables which contribute more to the discrimination and highlight which variables seem different between males and females.

In [ ]:
# Import the required python packages including 
# the custom Chemometric Model objects
import numpy as np
from sklearn import preprocessing
import pandas as pds
import matplotlib.pyplot as plt
import warnings
from sklearn.exceptions import DataConversionWarning

from pyChemometrics.ChemometricsPLSDA import ChemometricsPLSDA
from pyChemometrics.ChemometricsPCA import ChemometricsPCA
from pyChemometrics.ChemometricsPLS import ChemometricsPLS
from pyChemometrics.ChemometricsScaler import ChemometricsScaler
from pyChemometrics.plotting_utils import _scatterplots


# Use to obtain same values as in the text
np.random.seed(350)

The next cell sets up the figure display mode. The *notebook* mode allows interactive plotting.

In [ ]:
# Set the plot backend to support interactive plotting
%matplotlib notebook

## Data import

We will now import the LC-MS RPOS Dementia  dataset together with metadata required for this example.

rpos_x_matrix - LC-MS data matrix, with observations in rows and features as columns. The data has already been normalised by probabilistic quotient normalization to account for variation in urinary dilution.

variable_names - Unique names for each LC-MS feature. Each id is a concatenation of retention time (in seconds) and m/z (retentionTime_m/z)

#### Metadata
Gender - Biological sex of the participants (Male or female)

Age - Age of the study participants, in years

In [ ]:
# Load the dataset
dementia_rpos_dataset = pds.read_csv("./Data/Dementia_RPOS_XCMS.csv",delimiter=',')

rpos_x_matrix = dementia_rpos_dataset.iloc[:, 13::].values

variable_names = dementia_rpos_dataset.columns[13::]
# Use pandas Categorical type
gender_y = pds.Categorical(dementia_rpos_dataset['Gender']).codes

In [ ]:
# Extract the retention times and m/z to use in 2D plots of the dataset
retention_times = np.array([x.split('_')[0] for x in variable_names], dtype='float')/60
mz_values = np.array([x.split('_')[1][0:-3] for x in variable_names], dtype='float')

**Note**: To apply the analyses exemplified in this notebook to any other dataset, just modify the cell above to import the data matrices and vectors X and Y from any other source file.

The expected data types and formatting for **X** and **Y** are:

   **X**: Any data matrix with n rows (observations/samples) and p columns (variables/features). The matrix should be provided as a [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) object, with 2 dimensions, and with shape = (n, p). We recommend using the *numpy* function [numpy.genfromtxt](https://numpy.org/devdocs/reference/generated/numpy.genfromtxt.html) or the *pandas* [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function to read the data from a text file. When using the *pandas.read_csv* function, extract the data matrix as a *numpy.ndarray* from the pandas.DataFrame object using the `.values` attribute. 
```
X_DataFrame = pds.read_csv("./data/X_spectra.csv")
X = X_DataFrame.values
```
   
   **Y** vectors: Each **Y** vector should be a 1-dimensional [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) object, with a number and ordering of elements matching the rows in **X**. For continuous variables, any regular *numpy.ndarray* with a data type of `int` (integers only) or `float` can be used.
   ```
   Y_continuous = numpy.ndarray([23.4, 24, 0.3, -1.23], dtype='float')
   ```
To handle class labels, we recommend using the *pandas* [Categorical](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) datatype. After converting a column to a `Categorical` datatype, the `.codes` attribute returns a vector with the same length of the original Y, but where each value is replaced by their integer (`int`) code. The correspondence between code and category can be inspected with the `categories` attribute. The order of the labels in `.codes` is the same as the order of the `categories` attribute (i.e. 0 is the first element in `categories`, 1 the second and so on).
   ```
   Y1 = pds.Categorical(Y.iloc[:, 1])
   Y1.codes # The numerical label
   Y1.categories # Original text or numerical description of the category
   ```

Plot all the spectra in the dataset.


### Plotting the dataset 

The next cell will plot in 2-dimensions (retention time and m/z) the features on the reversed phased positive (RPOS) LC-MS dataset. High resolution LC-MS datasets are very complex, with numbers of features routinely exceeding > 10,000. Visualization of the raw dataset or even derived peak picked data sets is not straightforward. 2D scatter plots, such as the one shown below can be used to project quantities, of interest from multivariate and machine learning models, to get an overview of the co-elution patterns and m/z ranges from the detected signatures.

**Note**: 2D scatterplots contain a large number of datapoints. We recommend closing the plot after inspection by using the "power" button in the top right corner of the figure frame, to avoid performance issues when opening multiple plots.

In [ ]:
# Plot the spectra in the dataset
from pyChemometrics.plotting_utils import _scatterplots
# Helper 2D plot function from pyChemometrics
_scatterplots(np.log(np.mean(rpos_x_matrix, axis=0) + 1), xaxis=retention_times, yaxis=mz_values)

# PLS-DA modeling

## 1) Model fitting basics

In this section we will fit a PLS-DA model to classify urine LC-MS metabolic profiles based on the individual biological sex, and assess the metabolic differences in urine samples from males and females.

As an example, we start by fitting a PLS-DA model with 2 components and with mean-centring (MC) scaling. The choice of components to use in the modeling will be addressed properly in the next section, the objective of this first section is to introduce the model syntax.

Before mean centring, we will log-transform the data. One advantage of using log-transform in large and complex datasets is the effect log-transforming variables has on extreme (outlying values). After log-transformation, the impact of outliers on the fitted models (leverage) decreases []().

In [ ]:
# Select the scaling options: 
offset = np.min(rpos_x_matrix) + 1
log_X = np.log(rpos_x_matrix + offset)

# Mean centring (MC) scaling:
scaling_object_mc = ChemometricsScaler(scale_power=0)

In [ ]:
# Create and fit PLS-DA model
pls_da = ChemometricsPLSDA(n_components=2, x_scaler=scaling_object_mc)
pls_da.fit(log_X, gender_y)

PLS models perform dimensionality reduction in a manner similar to PCA. The main difference (besides the criteria in which the components are found) is that as well as the projections for the X matrix ($T$ scores) we also have projections for the Y matrix ($U$ scores).

Model visualization of PLS/PLS-DA models is typically performed by plotting the $T$ scores (X matrix scores). 
The score plot gives an overview of the relationships between samples, their similarities and dissimilatrities within the model space.

<br>

**Warning**: PLS-DA models can easily overfit, and the degree of separation or clustering of samples from distinct classes or Y outcome in the score plot is not a reliable measure of model validity. We recommend focusing on model validation before exploring the relationships in the scores plot. See the next section.

In [ ]:
# Plot the scores
pls_da.plot_scores(color=gender_y, discrete=True, label_outliers=True, plot_title=None)

Visualization of the loadings, weights and other model parameters are less straightforward in LC-MS datasets. We suggest using 2D scatterplots or barplots. 

In [ ]:
pls_da.plot_model_parameters(parameter='w', component=1, plottype='scatterplot', xaxis=retention_times, yaxis=mz_values)

In [ ]:
# Plot the weights and loadings.
# w for weights, p for loadings,
# ws for X rotations (rotated version of w) 
pls_da.plot_model_parameters(parameter='VIP', component=1, plottype='bar')

## 2) Model Selection - Number of components

Selection of the number of components for a PLS model follows a very similar logic to the PCA case.
Since the goal is to predict the Y variable, the main criteria used are the $R^{2}Y$/$Q^{2}Y$ as opposed to $R^{2}X$/$Q^{2}X$. 

However, the $R^{2}Y$/$Q^{2}Y$ measures are suited for regression problems, and not easily interpretable in classification problems. Metrics adequate for classification problems, such as accuracy or the Receiver-Operating Curve Area Under the Curve (ROC AUC) are more adequate in this context.

Ideally, we want to select enough components to predict as much of the variation in Y as possible using the data in X, while avoiding overfitting. 

We apply a similar criterion as the one used with PCA: choosing as the number of components after which the $Q^{2}Y$ or AUC value reaches a plateau (less than 5% increase compared to previous number of components). 

In [ ]:
pls_da.scree_plot(log_X, gender_y, total_comps=10)

Since this dataset has a large number of observations (561), we can further improve the robustness of the parameter selection and model evaluation by using a nested or [double cross-validation scheme](https://link.springer.com/article/10.1007/s11306-011-0330-3).

In a nested, or double cross-validation scheme the data is first split into a training and a test set, in an "outer" cv loop. The test set is kept aside. Then, the training test is used to optimize the model parameters. These are optimized not by taking the entire training set and fitting the model, but by using an "inner" cv loop, where the initial training data is again partitioned into independent training and test sets. Models with varying parameters are fitted in the "inner" training sets, and their performance benchmarked in the "inner" cv loop test sets. Then, parameters which give the best performance are selected, a model fitted on the entire training set. This model is tested on the test set kept aside in the "outer" cv loop. 

**Note**: Model cross-validation, especially *double cross validation* such as the one executed in the next cell requires fitting the model multiple times, and can take a few minutes. For reference, execution of the next cell takes (pproximately 15 mins on a desktop PC with 8 cores).
13min 7s ± 13.7 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# Repeated cross_validation
double_CvModel = pls_da.double_cross_validation(log_X, gender_y, total_comps=5)

### Inspect the model 
Inspect the model obtained using the number of components selected by double cross validation. 

In [ ]:
double_CvModel.plot_scores(color=pds.Categorical(dementia_rpos_dataset['Gender']).codes, discrete=True)

In [ ]:
print("Number of components selected by double cross-validation: {0}".format(double_CvModel.n_components))

To obtain more reliable estimates we can calculate the cross-validation estimates of any of these metrics, including cross-validated ROC curves. This ROC curve was estimated using the left-out samples (the test sets) during cross-validation.

In [ ]:
# Cross-validated ROC curve
double_CvModel.cross_validation(log_X, gender_y)
double_CvModel.plot_cv_ROC()

### Permutation Testing
A final and very important method for model validation is the permutation randomization test. In a permutation randomisation test, the model will be refitted and assessed multiple times, but each time with the Y randomly permuted to destroy any relationship between X & Y. This allows us to assess what sort of model we can get when there really is no relationship between the two data matrices, and calculate the likelihood of obtaining a model with predictive performance as good as the non-permuted model by chance alone.

During this test, the number of components, scaling, type of cross-validation employed, and any other modeling choice is kept constant. In each randomization, the model is refitted, and the AUC, $Q^{2}Y$ or any other validation metric is recorded. This enables the generation of permuted null distributions for any parameter, which can be used to obtain an empirical *p-value* for their significance.

**Note** Running the permutation test with a large number of permutation randomizations (for example, 1000) is expected to take a considerable ammount of time and might require leaving the computations running overnight (approximately 8h).

In [ ]:
permt = double_CvModel.permutation_test(log_X, gender_y, 10)

In [ ]:
# plot the results from the permuation test
double_CvModel.plot_permutation_test(permt, metric='AUC')
plt.xlabel('AUC')
plt.ylabel('Counts')
print("Permutation p-value for the AUC: {0}".format(permt[1]['AUC']))

The *p-value* obtained is < 0.05, so the model AUC and Q2Y values are significantly different from what is expected by chance alone at a level of $\alpha$ = 0.05.

## 3) Model interpretation and variable importance

The main parameters to assess in terms of variable importance for the prediction of Y from X are the weights ($w$), the VIP metric and regression coefficients.

The values in a weight vector vary between -1 (strong negative-covariance) and 1 (strong covariance), with 0 meaning no association/covariance. The weight vector of the first component (which explains the most variation in Y) is the primary weight vector to analyze when interpreting the main variables of X associed with Y.

The variable importance for prediction (VIP) metric is a sum (weighted by the ammount of variance of Y explained by each respective component) of the squared weight values. It provides a summary of the importance of a variable accounting for all weight vectors. VIPs are bounded between 0 (no effect) and infinity. Because it is calculated from the weights $w$, for PLS models with a single component these are directly proportional to the $w^{2}$. The VIP metric has the disadvantage of pooling together $w$ vectors from components which contribute a very small magnitude to the model's $R^{2}Y$.

The regression coefficients ($\beta$) have a similar interpretation as regression coefficients in a multivariate/multiple linear regression.

In [ ]:
# Save the model parameters and their cross-validated errors (standard deviation) to a file.
variableSelectionResults = pds.DataFrame(np.c_[variable_names, double_CvModel.weights_w[:, 0], double_CvModel.VIP(), double_CvModel.beta_coeffs, 
                                               double_CvModel.cvParameters['Stdev_Weights_w'][:, 0], double_CvModel.cvParameters['Stdev_VIP'], 
                                               double_CvModel.cvParameters['Stdev_Beta'].squeeze()], columns=['Feature Name', 'Weights_w - PLS Component 1', 'VIP', 'Beta', 'StDev_W', 'StDev_VIP', 'StDev_Beta'])
variableSelectionResults.to_csv('./Data/LC_MS_PLSDA_VarImportance.csv', index=False)

In [ ]:
double_CvModel.plot_model_parameters('w', component=1, plottype='scatterplot', 
                                     xaxis=retention_times, yaxis=mz_values)

In [ ]:
double_CvModel.plot_model_parameters('VIP', plottype='scatterplot', 
                                    xaxis=retention_times, yaxis=mz_values)

In [ ]:
double_CvModel.plot_model_parameters('beta', plottype='scatterplot', 
                                    xaxis=retention_times, yaxis=mz_values)

## Permutation p-values for variable ranking

The permutation test we ran before is also useful to obtain permuted null distributions for most of the model parameters. These can be used to obtain empirical confidence intervals and potentially permutation *p-values* for hypothesis testing.

We can now calculate empirical p-values for the regression coefficients...

In [ ]:
# Always set *nperms* equal to the number of permutations used before
nperms = permt[0]['R2Y'].size
perm_indx = abs(permt[0]['Beta'].squeeze()) >= abs(double_CvModel.beta_coeffs.squeeze())
counts = np.sum(perm_indx, axis=0)
beta_pvals = (counts + 1) / (nperms + 1)

perm_indx_W = abs(permt[0]['Weights_w'][:, :, 0].squeeze()) >= abs(double_CvModel.weights_w[:, 0].squeeze())
counts = np.sum(perm_indx_W, axis=0)
w_pvals = (counts + 1) / (nperms + 1)

perm_indx_vip = abs(permt[0]['VIPw']) >= abs(double_CvModel.VIP())
counts = np.sum(perm_indx_vip, axis=0)
vip_pvals = (counts + 1) / (nperms + 1)

In [ ]:
plt.figure()
plt.title(r"p-value distribution for the regression coefficients $\beta$ ")
z = plt.hist(beta_pvals, bins=100, alpha=0.8)
plt.axvline(x=0.05, ymin=0, ymax=max(z[0]), color='r', linestyle='--') 
plt.show()

plt.figure()
plt.title(r"p-value distribution for the weights corresponding to the first component")
z = plt.hist(w_pvals, bins=100, alpha=0.8)
plt.axvline(x=0.05, ymin=0, ymax=max(z[0]), color='r', linestyle='--') 
plt.show()

plt.figure()
plt.title(r"p-value distribution for the VIPw")
z = plt.hist(vip_pvals, bins=100, alpha=0.8)
plt.axvline(x=0.05, ymin=0, ymax=max(z[0]), color='r', linestyle='--') 
plt.show()

... and use the permutation test to obtain a list of statistically significant variables.

In [ ]:
signif_bpls_idx = np.where(beta_pvals <= 0.05)[0]

print("Number of significant values: {0}".format(len(signif_bpls_idx)))

signif_bpls_idx = np.where(vip_pvals <= 0.05)[0]

print("Number of significant values: {0}".format(len(signif_bpls_idx)))

signif_bpls_idx = np.where(w_pvals <= 0.05)[0]

print("Number of significant values: {0}".format(len(signif_bpls_idx)))

It is worth noting that a selection procedure of this kind is also a type of multiple testing, and it is recommended to apply false discovery rate or any other multiple testing correction to the *p-values* obtained in this manner. Also, formal inferential procedures to derive *p-values* and confidence intervals are not established for PLS models. Although *ad-hoc* solutions like a permutation test can be implemented as shown, some issues still remain - for example, the *p-value* distribution obtained for the regression coefficients is clearly non-uniform and care must be exercised when performing multiple testing correction or even interpreting the *p-values* obtained in this manner.

The latent variable and dimensionality reduction provided by PLS/PLS-DA can be very usefull to visualize general trends in the data. However, interpreting which variables are important to the model and how they contribute for the explanation/separation between classes is not easy. We suggest complementing the inspection of multivariate model parameters with univariate analysis.